## Machine Learning- Exoplanet Exploration

#### Extensive Data Dictionary: https://exoplanetarchive.ipac.caltech.edu/docs/API_kepcandidate_columns.html

Highlightable columns of note are:

* kepoi_name: A KOI is a target identified by the Kepler Project that displays at least one transit-like sequence within Kepler time-series photometry that appears to be of astrophysical origin and initially consistent with a planetary transit hypothesis

* kepler_name: [These names] are intended to clearly indicate a class of objects that have been confirmed or validated as planets—a step up from the planet candidate designation.

* koi_disposition: The disposition in the literature towards this exoplanet candidate. One of CANDIDATE, FALSE POSITIVE, NOT DISPOSITIONED or CONFIRMED.

* koi_pdisposition: The disposition Kepler data analysis has towards this exoplanet candidate. One of FALSE POSITIVE, NOT DISPOSITIONED, and CANDIDATE.

* koi_score: A value between 0 and 1 that indicates the confidence in the KOI disposition. For CANDIDATEs, a higher value indicates more confidence in its disposition, while for FALSE POSITIVEs, a higher value indicates less confidence in that disposition.


In [1]:
# # Update sklearn to prevent version mismatches
# !pip install sklearn --upgrade
# # install joblib
# !pip install joblib

### Import Dependencies 

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Hide warning messages in notebook
import warnings
warnings.filterwarnings('ignore')

# Read the CSV and Perform Basic Data Cleaning

In [3]:
# Read/Load CSV file
df = pd.read_csv("exoplanet_data.csv")

# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')

# Drop the null rows
df = df.dropna()
df.head()

,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,CONFIRMED,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,...,-81,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,FALSE POSITIVE,0,1,0,0,19.899140,1.490000e-05,-1.490000e-05,175.850252,0.000581,...,-176,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
2,FALSE POSITIVE,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,...,-174,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
3,CONFIRMED,0,0,0,0,2.525592,3.760000e-06,-3.760000e-06,171.595550,0.001130,...,-211,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
4,CONFIRMED,0,0,0,0,4.134435,1.050000e-05,-1.050000e-05,172.979370,0.001900,...,-232,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714


## Basic Statistic Details

In [4]:
df.describe()

,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,koi_time0bk_err2,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
count,6991.000000,6991.000000,6991.000000,6991.000000,6991.000000,6.991000e+03,6.991000e+03,6991.000000,6991.000000,6991.000000,...,6991.000000,6991.000000,6991.000000,6991.00000,6991.000000,6991.000000,6991.000000,6991.000000,6991.000000,6991.000000
mean,0.157059,0.244743,0.202975,0.125018,56.191248,1.851122e-03,-1.851122e-03,164.488820,0.009340,-0.009340,...,-161.206980,4.305049,0.121091,-0.14048,1.740749,0.352710,-0.388568,292.082406,43.812143,14.271508
std,0.363882,0.429966,0.402243,0.330763,117.570962,7.184503e-03,7.184503e-03,67.020475,0.021989,0.021989,...,71.448481,0.439238,0.132048,0.08199,5.903415,0.839017,1.907797,4.762908,3.606167,1.350802
min,0.000000,0.000000,0.000000,0.000000,0.259820,1.100000e-08,-1.568000e-01,120.515914,0.000009,-0.569000,...,-1733.000000,0.047000,0.000000,-1.00700,0.109000,0.000000,-103.825000,279.856080,36.577381,6.966000
25%,0.000000,0.000000,0.000000,0.000000,2.620126,5.005000e-06,-2.401000e-04,132.683917,0.001145,-0.010000,...,-197.000000,4.209000,0.044000,-0.19500,0.829000,0.128000,-0.252000,288.704730,40.797760,13.455000
50%,0.000000,0.000000,0.000000,0.000000,8.947426,3.300000e-05,-3.300000e-05,136.739230,0.003990,-0.003990,...,-159.000000,4.436000,0.070000,-0.12700,0.999000,0.248000,-0.111000,292.314760,43.679661,14.534000
75%,0.000000,0.000000,0.000000,0.000000,34.282605,2.401000e-04,-5.005000e-06,169.937005,0.010000,-0.001145,...,-112.000000,4.543000,0.149000,-0.08800,1.357000,0.357000,-0.069000,295.888550,46.693659,15.322000
max,1.000000,1.000000,1.000000,1.000000,1071.232624,1.568000e-01,-1.100000e-08,1472.522306,0.569000,-0.000009,...,0.000000,5.364000,1.472000,0.00000,180.013000,25.956000,0.000000,301.720760,52.336010,19.065000


# Select Features (columns)
* Feature Selection: Removing irrelevant feature results in better performing model that is easeir to understands & model runs faster


In [24]:
target_names = df["koi_disposition"].unique()
#target_names
print(df["koi_disposition"].unique())

['CONFIRMED' 'FALSE POSITIVE' 'CANDIDATE']


In [25]:
# Assign X (Independant data) and y (Dependant target)

# Set X equal to the entire data set, except for the first column
X = df.iloc[:, 1:]
# X.head()

# Set y equal to the first column
y = df.iloc[:,0].values.reshape(-1, 1)
# y.head()

In [26]:
from sklearn.ensemble import ExtraTreesClassifier

# Search for top 10 features according to feature importances
model = ExtraTreesClassifier()
model.fit(X,y)
model.feature_importances_

# sorted(zip(model.feature_importances_, X), reverse=True)

array([0.15511083, 0.17745117, 0.12265767, 0.05688023, 0.01793089,
       0.01328461, 0.00987712, 0.01102556, 0.01643729, 0.01767205,
       0.01061325, 0.01110032, 0.00867455, 0.01960285, 0.03957704,
       0.02528608, 0.01676593, 0.0101311 , 0.0101305 , 0.01225717,
       0.00896745, 0.0091171 , 0.01423994, 0.00863058, 0.00862274,
       0.00883035, 0.03168312, 0.00889892, 0.01031239, 0.01723097,
       0.01225381, 0.01289126, 0.01046478, 0.01367504, 0.00977122,
       0.00932827, 0.00977346, 0.01247569, 0.01033136, 0.01003539])

In [27]:
# Store the top (10) features  as a series, using the column headers as the index
top_feat = pd.Series(model.feature_importances_, index=X.columns).nlargest(10)
top_feat

koi_fpflag_ss        0.177451
koi_fpflag_nt        0.155111
koi_fpflag_co        0.122658
koi_fpflag_ec        0.056880
koi_duration_err1    0.039577
koi_model_snr        0.031683
koi_duration_err2    0.025286
koi_duration         0.019603
koi_period           0.017931
koi_time0bk_err2     0.017672
dtype: float64

In [71]:
# Set features based on feature importances
X = df[top_feat.index]

# Use `koi_disposition` for the y values
y = df['koi_disposition']

#y = df['koi_disposition'].values.reshape(-1, 1)

# Create a Train Test Split

In [72]:
from sklearn.model_selection import train_test_split

# Split the data into smaller buckets for training and testing
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
X_train.head()

,koi_fpflag_ss,koi_fpflag_nt,koi_fpflag_co,koi_fpflag_ec,koi_duration_err1,koi_model_snr,koi_duration_err2,koi_duration,koi_period,koi_time0bk_err2
6122,0,0,0,0,0.3060,10.8,-0.3060,3.61600,6.768901,-0.008440
6370,1,0,0,1,0.2820,13.8,-0.2820,2.30900,0.733726,-0.007950
2879,0,1,0,0,0.0000,254.3,0.0000,79.89690,7.652707,-0.006190
107,0,0,0,0,0.0595,38.4,-0.0595,2.63120,7.953547,-0.001820
29,0,0,0,0,0.0075,696.5,-0.0075,2.22739,4.959319,-0.000083


In [73]:
# X and y Train shape have 5243 rows (80% of data)
X_train.shape, y_train.shape

((5243, 10), (5243,))

In [74]:
# X and y Test shape have 1748 rows (20% of data)
X_test.shape, y_test.shape

((1748, 10), (1748,))

## Accuracy

In [75]:
# Predicting the Test set results
y_predic = model.predict(X_test)

# Making the confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_predic)

from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, y_predic)

accuracy

0.7299771167048055

# Pre-processing

Scale the data using the MinMaxScaler

MinMaxScaler: 
              * A way to normalize the input features/variables
              * Features will be transformed into the range
              * Scales the range of fetures from 0 to 1


In [76]:
from sklearn.preprocessing import MinMaxScaler

# Create a MinMaxScaler model and fit it to the training data
X_scaler = MinMaxScaler().fit(X_train)

#Transform the training and testing data using the X_scaler
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)
#print(np.matrix(X_test_scaled))

# Train the Model 
* Used Logistic Regression Model



In [77]:
from sklearn.linear_model import LogisticRegression

# Create a Logistic Regression Model
model = LogisticRegression()

# Train (Fit) the model to the data
model.fit(X_train_scaled, y_train)

# Validate the model using the test data
print(f"Training Data Score: {model.score(X_train_scaled, y_train)}")
print(f"Testing Data Score: {model.score(X_test_scaled, y_test)}")

# Printed the r2 score for the test data, testing is lower than training which is good we are not over feeding

Training Data Score: 0.8088880411977876
Testing Data Score: 0.8106407322654462


## Prediction

In [78]:
predictions = model.predict(X_test_scaled)

print(f"first 10 Predictions{predictions[:10].tolist()}")
print(f"first 10 Actual{y_test[:10].tolist()}")

#Printing into a Dataframe (y_test can't be reshap on top)
# df_pred = pd.DataFrame({"Actual":y_test, "Predicted":predictions}) 
# df_pred.head()

first 10 Predictions['FALSE POSITIVE', 'CANDIDATE', 'FALSE POSITIVE', 'FALSE POSITIVE', 'FALSE POSITIVE', 'CONFIRMED', 'CONFIRMED', 'CANDIDATE', 'CANDIDATE', 'FALSE POSITIVE']
first 10 Actual['FALSE POSITIVE', 'CANDIDATE', 'FALSE POSITIVE', 'FALSE POSITIVE', 'FALSE POSITIVE', 'CONFIRMED', 'CANDIDATE', 'CANDIDATE', 'CANDIDATE', 'FALSE POSITIVE']


# Hyperparameter Tuning

Use `GridSearchCV` to tune the model's parameters

In [79]:
from sklearn.svm import SVC

# Create the GridSearchCV model
model = SVC(kernel='linear')
model.fit(X_train_scaled, y_train)

# Predicting the Test set results
y_predic = model.predict(X_test)

# Making the confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_predic)

from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, y_predic)

accuracy

0.7299771167048055

In [80]:
from sklearn.model_selection import GridSearchCV

param_grid = {'C': [1, 5, 10, 20, 50],
              'gamma': [0.0001,0.001, 0.01, 0.0005, 0.005]}

grid = GridSearchCV(model, param_grid, verbose=3)


In [81]:
# Train the model with GridSearch

grid.fit(X_train_scaled, y_train)

Fitting 3 folds for each of 25 candidates, totalling 75 fits
[CV] C=1, gamma=0.0001 ...............................................
[CV] ................... C=1, gamma=0.0001, score=0.790, total=   0.1s
[CV] C=1, gamma=0.0001 ...............................................
[CV] ................... C=1, gamma=0.0001, score=0.783, total=   0.1s
[CV] C=1, gamma=0.0001 ...............................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.1s remaining:    0.0s


[CV] ................... C=1, gamma=0.0001, score=0.791, total=   0.1s
[CV] C=1, gamma=0.001 ................................................
[CV] .................... C=1, gamma=0.001, score=0.790, total=   0.1s
[CV] C=1, gamma=0.001 ................................................
[CV] .................... C=1, gamma=0.001, score=0.783, total=   0.1s
[CV] C=1, gamma=0.001 ................................................
[CV] .................... C=1, gamma=0.001, score=0.791, total=   0.1s
[CV] C=1, gamma=0.01 .................................................
[CV] ..................... C=1, gamma=0.01, score=0.790, total=   0.1s
[CV] C=1, gamma=0.01 .................................................
[CV] ..................... C=1, gamma=0.01, score=0.783, total=   0.1s
[CV] C=1, gamma=0.01 .................................................
[CV] ..................... C=1, gamma=0.01, score=0.791, total=   0.1s
[CV] C=1, gamma=0.0005 ...............................................
[CV] .

[CV] .................. C=50, gamma=0.0001, score=0.877, total=   0.1s
[CV] C=50, gamma=0.0001 ..............................................
[CV] .................. C=50, gamma=0.0001, score=0.872, total=   0.1s
[CV] C=50, gamma=0.0001 ..............................................
[CV] .................. C=50, gamma=0.0001, score=0.858, total=   0.1s
[CV] C=50, gamma=0.001 ...............................................
[CV] ................... C=50, gamma=0.001, score=0.877, total=   0.1s
[CV] C=50, gamma=0.001 ...............................................
[CV] ................... C=50, gamma=0.001, score=0.872, total=   0.1s
[CV] C=50, gamma=0.001 ...............................................
[CV] ................... C=50, gamma=0.001, score=0.858, total=   0.1s
[CV] C=50, gamma=0.01 ................................................
[CV] .................... C=50, gamma=0.01, score=0.877, total=   0.1s
[CV] C=50, gamma=0.01 ................................................
[CV] .

[Parallel(n_jobs=1)]: Done  75 out of  75 | elapsed:    6.1s finished


GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='auto_deprecated', kernel='linear',
                           max_iter=-1, probability=False, random_state=None,
                           shrinking=True, tol=0.001, verbose=False),
             iid='warn', n_jobs=None,
             param_grid={'C': [1, 5, 10, 20, 50],
                         'gamma': [0.0001, 0.001, 0.01, 0.0005, 0.005]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=3)

In [82]:
# List the best parameters for this dataset
print(grid.best_params_)

# List the best score
print('Best Score: ', grid.best_score_)

{'C': 50, 'gamma': 0.0001}
Best Score:  0.8689681480068663


In [83]:
# Score the model
grid.score(X_test_scaled, y_test)

0.8667048054919908

In [84]:
# Make predictions with the hypertuned model
predictions = grid.predict(X_test_scaled)
df_grid = pd.DataFrame({"Actual":y_test, "Predicted":predictions}) 
df_grid.head()

,Actual,Predicted
4982,FALSE POSITIVE,FALSE POSITIVE
4866,CANDIDATE,CANDIDATE
2934,FALSE POSITIVE,FALSE POSITIVE
5007,FALSE POSITIVE,FALSE POSITIVE
3869,FALSE POSITIVE,FALSE POSITIVE


In [85]:
# Calculate classification report
# print(np.array(y_test))

from sklearn.metrics import classification_report
print(classification_report(y_test, predictions,
                            target_names=target_names))

                precision    recall  f1-score   support

     CONFIRMED       0.83      0.58      0.68       411
FALSE POSITIVE       0.72      0.88      0.79       484
     CANDIDATE       0.98      1.00      0.99       853

      accuracy                           0.87      1748
     macro avg       0.84      0.82      0.82      1748
  weighted avg       0.87      0.87      0.86      1748



# Save the Model

In [86]:
import joblib
filename = 'LogisticRegression.sav'
joblib.dump(LogisticRegression, filename)

['LogisticRegression.sav']